# 🎯 Prompt Templates e OutputParsers: A Arte de Conversar com IA de Forma Profissional!

**Módulo 3 - LangChain v0.2 | Por Pedro Guth**

---

E aí pessoal! 🚀 Bem-vindos ao terceiro módulo do nosso curso de LangChain!

No módulo anterior, aprendemos sobre ChatModels e LCEL. Agora vamos dar um passo GIGANTE para frente: aprender a criar prompts profissionais e parsear as respostas da IA de forma estruturada!

Tá, mas Pedro, por que isso é importante? 🤔

Imagina que você tem um assistente super inteligente, mas ele só entende instruções muito específicas e às vezes responde de forma bagunçada. Os **Prompt Templates** são como um roteiro bem estruturado, e os **OutputParsers** são como um tradutor que organiza tudo bonitinho!

**Analogia do Garçom:** É como a diferença entre chegar no restaurante e falar "quero comida" vs "Oi, gostaria do prato executivo com frango grelhado, arroz branco e salada verde, por favor". Qual vai dar melhor resultado? 😄

## 📋 O que vamos aprender hoje?

1. **Prompt Templates**: Como criar prompts reutilizáveis e profissionais
2. **OutputParsers**: Como estruturar as respostas da IA
3. **Integração**: Como combinar tudo isso com LCEL
4. **Casos práticos**: Exemplos do mundo real
5. **Preparação**: Base para Chains (próximo módulo)

---

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-03_img_01.png)

In [ ]:
# Vamos começar instalando e importando tudo que precisamos!
# Bora preparar nosso ambiente de trabalho! 🛠️

!pip install langchain langchain-google-genai python-dotenv matplotlib

import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser, CommaSeparatedListOutputParser
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field
from typing import List
import matplotlib.pyplot as plt
import json

# Carregando as variáveis de ambiente
load_dotenv()

print("✅ Imports realizados com sucesso!")
print("🚀 Vamos começar nossa jornada com Prompt Templates!")

In [ ]:
# Configurando nosso modelo (lembrando do módulo anterior!)
# Usando o Gemini 2.0 Flash como definido no curso

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.7,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Testando se está funcionando
resposta_simples = llm.invoke("Olá! Como você está?")
print("🤖 Teste do modelo:")
print(resposta_simples.content)

print("\n✅ Modelo configurado e funcionando!")

## 🎭 Entendendo Prompt Templates

**Tá, mas o que são Prompt Templates mesmo?**

Pense nos Prompt Templates como **formulários inteligentes** para conversar com IA. Em vez de escrever prompts do zero toda vez, você cria um "molde" que pode ser reutilizado!

**Analogia da Receita de Bolo:** 🍰
- Sem template: "Faça um bolo" (muito vago!)
- Com template: "Faça um bolo de {sabor} com {cobertura}, serve {pessoas} pessoas" (específico e reutilizável!)

### Vantagens dos Prompt Templates:

1. **Consistência**: Sempre o mesmo padrão
2. **Reutilização**: Muda só as variáveis
3. **Manutenção**: Altera em um lugar, funciona em todo lugar
4. **Profissionalismo**: Prompts bem estruturados



In [ ]:
# Exemplo prático: Prompt sem template vs com template
# Vamos ver a diferença na prática!

# ❌ JEITO RUIM - Sem template (sempre recriando)
prompt_ruim_1 = "Escreva um email de marketing para vender curso de Python"
prompt_ruim_2 = "Escreva um email de marketing para vender curso de Java"
prompt_ruim_3 = "Escreva um email de marketing para vender curso de JavaScript"

print("❌ Jeito ruim - Repetindo código:")
print(f"Prompt 1: {prompt_ruim_1}")
print(f"Prompt 2: {prompt_ruim_2}")
print(f"Prompt 3: {prompt_ruim_3}")

print("\n" + "="*50 + "\n")

# ✅ JEITO BOM - Com template
template_bom = PromptTemplate(
    input_variables=["curso", "nivel"],
    template="""
    Escreva um email de marketing profissional para vender um curso de {curso}.
    
    Detalhes:
    - Nível: {nivel}
    - Tom: Entusiasta mas profissional
    - Tamanho: Máximo 200 palavras
    - Inclua call-to-action
    
    Email:
    """
)

print("✅ Jeito bom - Com template reutilizável:")
print(template_bom.template)

# Testando o template
prompt_python = template_bom.format(curso="Python", nivel="Iniciante")
print(f"\n🐍 Prompt para Python:\n{prompt_python}")

### 💡 Dica do Pedro:

**Sempre use templates quando:**
- O prompt tem estrutura repetitiva
- Você vai usar o mesmo padrão várias vezes
- Quer manter consistência no projeto
- Está trabalhando em equipe

---

## 🔄 Tipos de Prompt Templates

O LangChain oferece diferentes tipos de templates para diferentes necessidades:

In [ ]:
# 1. PromptTemplate - Para modelos de texto simples
template_simples = PromptTemplate(
    input_variables=["produto", "problema"],
    template="Como o {produto} pode resolver o problema de {problema}?"
)

print("1️⃣ PromptTemplate Simples:")
print(template_simples.format(produto="ChatGPT", problema="produtividade no trabalho"))

print("\n" + "-"*50 + "\n")

# 2. ChatPromptTemplate - Para modelos de chat (mais moderno!)
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um especialista em {area} com 10 anos de experiência."),
    ("human", "Me explique sobre {topico} de forma simples e prática.")
])

print("2️⃣ ChatPromptTemplate (RECOMENDADO):")
mensagens = chat_template.format_messages(
    area="Inteligência Artificial",
    topico="Redes Neurais"
)

for msg in mensagens:
    print(f"{msg.type}: {msg.content}")

print("\n" + "-"*50 + "\n")

# 3. Template com múltiplas variáveis e lógica
template_avancado = ChatPromptTemplate.from_template("""
Você é um {papel} especializado em {area}.

Tarefa: {tarefa}

Contexto:
- Público-alvo: {publico}
- Nível de conhecimento: {nivel}
- Formato desejado: {formato}

Instruções específicas:
1. Use linguagem {tom}
2. Inclua exemplos práticos
3. Mantenha o foco em {foco}

Resposta:
""")

print("3️⃣ Template Avançado com Múltiplas Variáveis:")
print("Template criado com sucesso! ✅")

## 📊 Visualizando o Fluxo de Prompt Templates

Vamos criar um diagrama para entender melhor como funciona:

In [ ]:
# Vamos visualizar o processo de criação e uso de templates
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch

fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Definindo as etapas
etapas = [
    ("1. Template\nDefinition", 1, 6, "lightblue"),
    ("2. Variable\nInput", 3, 6, "lightgreen"),
    ("3. Format\nPrompt", 5, 6, "lightyellow"),
    ("4. Send to\nLLM", 7, 6, "lightcoral"),
    ("5. Get\nResponse", 9, 6, "lightpink")
]

# Desenhando as caixas
for texto, x, y, cor in etapas:
    box = FancyBboxPatch((x-0.4, y-0.4), 0.8, 0.8, 
                         boxstyle="round,pad=0.1", 
                         facecolor=cor, 
                         edgecolor='black',
                         linewidth=2)
    ax.add_patch(box)
    ax.text(x, y, texto, ha='center', va='center', fontsize=10, fontweight='bold')

# Desenhando as setas
for i in range(len(etapas)-1):
    x1 = etapas[i][1] + 0.4
    x2 = etapas[i+1][1] - 0.4
    y = 6
    ax.annotate('', xy=(x2, y), xytext=(x1, y),
                arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))

# Adicionando exemplos embaixo
exemplos = [
    "Template:\n'Explique {topico}\npara {publico}'",
    "Variables:\ntopico='AI'\npublico='crianças'",
    "Formatted:\n'Explique AI\npara crianças'",
    "LLM\nProcessing",
    "Response:\n'AI é como um\nrobô inteligente...'"
]

for i, (exemplo, x, y, cor) in enumerate(zip(exemplos, [e[1] for e in etapas], [4]*5, [e[3] for e in etapas])):
    ax.text(x, y, exemplo, ha='center', va='center', fontsize=8, 
            bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))

ax.set_xlim(0, 10)
ax.set_ylim(3, 7)
ax.set_title('🎯 Fluxo de Prompt Templates no LangChain', fontsize=16, fontweight='bold', pad=20)
ax.axis('off')

plt.tight_layout()
plt.show()

print("📊 Fluxo visualizado! Agora você entende o processo completo!")

## 🎯 Output Parsers - Organizando as Respostas

**Tá, Pedro, mas o que são Output Parsers?**

Imagina que você pergunta para um amigo "Me fala 3 filmes bons" e ele responde:

*"Ah cara, tem aquele filme do Batman que é muito bom, sabe? E também O Poderoso Chefão que é clássico né, ah e Interestelar que é meio complexo mas é demais!"*

Os **Output Parsers** são como um organizador que pega essa resposta bagunçada e transforma em:
```
1. Batman
2. O Poderoso Chefão  
3. Interestelar
```

**Em termos técnicos:** Output Parsers pegam a resposta de texto da IA e transformam em estruturas de dados úteis (listas, dicionários, objetos, etc.)

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-03_img_03.png)

In [ ]:
# Vamos ver na prática a diferença!
# Exemplo: Pedindo uma lista de linguagens de programação

# ❌ SEM Output Parser - Resposta bagunçada
template_sem_parser = ChatPromptTemplate.from_template(
    "Liste 5 linguagens de programação populares."
)

chain_sem_parser = template_sem_parser | llm
resposta_bagunçada = chain_sem_parser.invoke({})

print("❌ SEM Output Parser - Resposta bagunçada:")
print(resposta_bagunçada.content)
print(f"Tipo da resposta: {type(resposta_bagunçada.content)}")

print("\n" + "="*60 + "\n")

# ✅ COM Output Parser - Resposta estruturada
list_parser = CommaSeparatedListOutputParser()

template_com_parser = ChatPromptTemplate.from_template(
    "Liste 5 linguagens de programação populares.\n{format_instructions}"
)

# O parser automaticamente adiciona instruções de formatação!
print("✅ COM Output Parser - Instruções automáticas:")
print("Instruções geradas pelo parser:")
print(list_parser.get_format_instructions())

# Criando a chain completa
chain_com_parser = template_com_parser | llm | list_parser

# Executando
resposta_organizada = chain_com_parser.invoke({
    "format_instructions": list_parser.get_format_instructions()
})

print("\nResposta estruturada:")
print(resposta_organizada)
print(f"Tipo da resposta: {type(resposta_organizada)}")
print(f"É uma lista? {isinstance(resposta_organizada, list)}")

### 💡 Dica do Pedro:

**Por que usar Output Parsers?** 🤔

1. **Previsibilidade**: Sempre recebe dados no formato esperado
2. **Integração**: Fácil de usar com código Python
3. **Validação**: Garante que os dados estão corretos
4. **Produtividade**: Não precisa processar texto manualmente

---

## 🛠️ Tipos de Output Parsers

In [ ]:
# 1. CommaSeparatedListOutputParser - Para listas simples
print("1️⃣ CommaSeparatedListOutputParser")
print("-" * 40)

list_parser = CommaSeparatedListOutputParser()
template_lista = ChatPromptTemplate.from_template(
    "Liste 3 benefícios de aprender Python.\n{format_instructions}"
)

chain_lista = template_lista | llm | list_parser
resultado_lista = chain_lista.invoke({
    "format_instructions": list_parser.get_format_instructions()
})

print(f"Resultado: {resultado_lista}")
print(f"Tipo: {type(resultado_lista)}")
print(f"Primeiro item: {resultado_lista[0]}")

print("\n" + "="*50 + "\n")

# 2. Vamos criar um parser customizado simples
class SimpleJSONParser(BaseOutputParser):
    """Parser customizado para JSON simples"""
    
    def get_format_instructions(self) -> str:
        return "Responda em formato JSON válido com as chaves especificadas."
    
    def parse(self, text: str) -> dict:
        # Remove possíveis caracteres extras e extrai o JSON
        try:
            # Procura por { até } no texto
            start = text.find('{')
            end = text.rfind('}') + 1
            if start != -1 and end != 0:
                json_str = text[start:end]
                return json.loads(json_str)
            else:
                raise ValueError("JSON não encontrado")
        except Exception as e:
            return {"erro": f"Não foi possível parsear: {str(e)}", "texto_original": text}

print("2️⃣ Parser Customizado - SimpleJSONParser")
print("-" * 40)

json_parser = SimpleJSONParser()
template_json = ChatPromptTemplate.from_template(
    "Crie um perfil de desenvolvedor Python com nome, experiência (anos) e especialidade.\n{format_instructions}"
)

chain_json = template_json | llm | json_parser
resultado_json = chain_json.invoke({
    "format_instructions": json_parser.get_format_instructions()
})

print(f"Resultado JSON: {resultado_json}")
print(f"Tipo: {type(resultado_json)}")

if isinstance(resultado_json, dict) and "nome" in resultado_json:
    print(f"Nome do dev: {resultado_json.get('nome', 'N/A')}")

## 🏗️ Pydantic Output Parser - O Mais Poderoso!

**Tá, mas Pedro, e se eu quiser algo mais robusto?**

Aí entra o **PydanticOutputParser**! É como ter um assistente super exigente que verifica se tudo está certinho antes de entregar o resultado.

**Analogia do Controle de Qualidade:** 🏭
É como uma fábrica que não só organiza os produtos, mas também verifica se cada um tem as especificações corretas antes de sair da linha de produção!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-03_img_04.png)

In [ ]:
# Definindo um modelo Pydantic - Estrutura de dados com validação
class PerfilProgramador(BaseModel):
    """Modelo para perfil de programador"""
    nome: str = Field(description="Nome completo do programador")
    idade: int = Field(description="Idade em anos", ge=18, le=80)  # ge = greater equal, le = less equal
    linguagens: List[str] = Field(description="Lista de linguagens que domina")
    experiencia_anos: int = Field(description="Anos de experiência", ge=0)
    salario_desejado: float = Field(description="Salário desejado em reais")
    remoto: bool = Field(description="Aceita trabalho remoto")
    especialidade: str = Field(description="Área de especialização")

# Criando o parser
pydantic_parser = PydanticOutputParser(pydantic_object=PerfilProgramador)

print("🏗️ PydanticOutputParser - Estrutura Definida:")
print("Campos obrigatórios:")
for campo, info in PerfilProgramador.__fields__.items():
    print(f"  - {campo}: {info.annotation}")

print("\n📋 Instruções automáticas geradas:")
print(pydantic_parser.get_format_instructions())

In [ ]:
# Usando o PydanticOutputParser na prática
template_pydantic = ChatPromptTemplate.from_template(
    """
    Crie um perfil fictício de um programador brasileiro.
    
    Seja realista com os valores e informações.
    Use dados que fazem sentido para o mercado brasileiro.
    
    {format_instructions}
    """
)

# Criando a chain completa
chain_pydantic = template_pydantic | llm | pydantic_parser

print("🚀 Executando chain com Pydantic Parser...")

try:
    resultado_pydantic = chain_pydantic.invoke({
        "format_instructions": pydantic_parser.get_format_instructions()
    })
    
    print("✅ Sucesso! Perfil criado:")
    print(f"📋 Tipo do resultado: {type(resultado_pydantic)}")
    print(f"👤 Nome: {resultado_pydantic.nome}")
    print(f"🎂 Idade: {resultado_pydantic.idade} anos")
    print(f"💻 Linguagens: {', '.join(resultado_pydantic.linguagens)}")
    print(f"⏱️ Experiência: {resultado_pydantic.experiencia_anos} anos")
    print(f"💰 Salário desejado: R$ {resultado_pydantic.salario_desejado:,.2f}")
    print(f"🏠 Trabalho remoto: {'Sim' if resultado_pydantic.remoto else 'Não'}")
    print(f"🎯 Especialidade: {resultado_pydantic.especialidade}")
    
    # Mostrando que é um objeto Python real!
    print(f"\n🔍 Acessando como objeto Python:")
    print(f"Primeira linguagem: {resultado_pydantic.linguagens[0]}")
    print(f"É sênior? {resultado_pydantic.experiencia_anos >= 5}")
    
except Exception as e:
    print(f"❌ Erro: {e}")
    print("Isso pode acontecer se a IA não seguir exatamente o formato!")

## 📊 Comparando Diferentes Output Parsers

Vamos criar uma visualização para comparar os diferentes tipos:

In [ ]:
# Comparando performance e complexidade dos parsers
import matplotlib.pyplot as plt
import numpy as np

parsers = ['Sem Parser', 'Lista Simples', 'JSON Custom', 'Pydantic']
complexidade = [1, 3, 6, 9]
validacao = [0, 2, 5, 10]
reutilizacao = [2, 5, 7, 10]

x = np.arange(len(parsers))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 8))

# Criando as barras
rects1 = ax.bar(x - width, complexidade, width, label='Complexidade', color='lightcoral', alpha=0.8)
rects2 = ax.bar(x, validacao, width, label='Validação', color='lightblue', alpha=0.8)
rects3 = ax.bar(x + width, reutilizacao, width, label='Reutilização', color='lightgreen', alpha=0.8)

# Adicionando labels e título
ax.set_xlabel('Tipos de Output Parsers', fontsize=12, fontweight='bold')
ax.set_ylabel('Pontuação (0-10)', fontsize=12, fontweight='bold')
ax.set_title('🔍 Comparação de Output Parsers\n(Complexidade vs Benefícios)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(parsers)
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Adicionando valores nas barras
def adicionar_valores(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontweight='bold')

adicionar_valores(rects1)
adicionar_valores(rects2)
adicionar_valores(rects3)

plt.tight_layout()
plt.show()

# Tabela explicativa
print("📊 Análise dos Output Parsers:")
print("-" * 50)
print("🔴 Sem Parser:")
print("  + Simples de implementar")
print("  - Sem validação, dados inconsistentes")
print("\n🟡 Lista Simples:")
print("  + Fácil para listas básicas")
print("  - Limitado a estruturas simples")
print("\n🟠 JSON Custom:")
print("  + Flexível e customizável")
print("  - Requer mais código para validação")
print("\n🟢 Pydantic (RECOMENDADO):")
print("  + Validação automática")
print("  + Type hints")
print("  + Documentação automática")
print("  + Reutilizável")
print("  - Mais complexo para casos simples")

## 🔗 Integrando Templates e Parsers com LCEL

**Agora vem a parte LINDA! 🎨**

Vamos combinar tudo que aprendemos usando LCEL (LangChain Expression Language) que vimos no módulo anterior!

**Analogia da Linha de Produção:** 🏭
É como uma fábrica moderna onde cada etapa está perfeitamente sincronizada:
1. **Matéria-prima** (seus dados) → 
2. **Molde** (Template) → 
3. **Máquina** (LLM) → 
4. **Controle de Qualidade** (Parser) → 
5. **Produto Final** (dados estruturados)

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-03_img_05.png)

In [ ]:
# Exemplo prático: Sistema de análise de produtos
# Vamos criar um sistema que analisa produtos e retorna dados estruturados

# 1. Definindo o modelo de dados
class AnalyseProduto(BaseModel):
    """Análise estruturada de produto"""
    nome_produto: str = Field(description="Nome do produto")
    categoria: str = Field(description="Categoria do produto")
    pontos_fortes: List[str] = Field(description="Lista de pontos fortes")
    pontos_fracos: List[str] = Field(description="Lista de pontos fracos")
    nota_geral: int = Field(description="Nota de 1 a 10", ge=1, le=10)
    publico_alvo: str = Field(description="Público-alvo principal")
    preco_justo: float = Field(description="Preço justo estimado em reais")
    recomenda: bool = Field(description="Recomenda a compra")

# 2. Criando o parser
analise_parser = PydanticOutputParser(pydantic_object=AnalyseProduto)

# 3. Criando o template
analise_template = ChatPromptTemplate.from_template(
    """
    Você é um especialista em análise de produtos de tecnologia.
    
    Analise o seguinte produto: {produto}
    
    Categoria do produto: {categoria}
    
    Faça uma análise completa e honesta, considerando:
    - Qualidade e funcionalidades
    - Custo-benefício
    - Público-alvo
    - Comparação com concorrentes
    - Preço justo para o mercado brasileiro
    
    Seja específico e realista nas suas avaliações.
    
    {format_instructions}
    """
)

# 4. Criando a chain completa com LCEL
analise_chain = analise_template | llm | analise_parser

print("🏭 Sistema de Análise de Produtos Criado!")
print("✅ Template: Configurado")
print("✅ Parser: Configurado")
print("✅ Chain: Pronta para uso")

# Testando o sistema
print("\n🧪 Testando com iPhone 15...")

try:
    resultado = analise_chain.invoke({
        "produto": "iPhone 15",
        "categoria": "Smartphone",
        "format_instructions": analise_parser.get_format_instructions()
    })
    
    print("\n📱 ANÁLISE COMPLETA:")
    print("=" * 50)
    print(f"📦 Produto: {resultado.nome_produto}")
    print(f"📂 Categoria: {resultado.categoria}")
    print(f"⭐ Nota: {resultado.nota_geral}/10")
    print(f"💰 Preço justo: R$ {resultado.preco_justo:,.2f}")
    print(f"🎯 Público-alvo: {resultado.publico_alvo}")
    print(f"✅ Recomenda: {'Sim' if resultado.recomenda else 'Não'}")
    
    print(f"\n👍 Pontos Fortes:")
    for i, ponto in enumerate(resultado.pontos_fortes, 1):
        print(f"  {i}. {ponto}")
    
    print(f"\n👎 Pontos Fracos:")
    for i, ponto in enumerate(resultado.pontos_fracos, 1):
        print(f"  {i}. {ponto}")
        
except Exception as e:
    print(f"❌ Erro na análise: {e}")

### 💡 Dica do Pedro - Melhores Práticas:

1. **Sempre use ChatPromptTemplate** para modelos modernos
2. **PydanticOutputParser para dados estruturados** importantes
3. **Trate erros** - nem sempre a IA segue o formato perfeitamente
4. **Teste com dados reais** antes de colocar em produção
5. **Use LCEL** para chains mais limpas e legíveis

---

## 🎮 Exercício Prático 1 - Seu Primeiro Sistema Completo

In [ ]:
# 🎯 EXERCÍCIO: Crie um sistema de recomendação de cursos
# 
# Sua missão:
# 1. Criar um modelo Pydantic para RecomendacaoCurso
# 2. Criar um template que recebe: interesse, nivel, tempo_disponivel
# 3. Criar uma chain completa
# 4. Testar com seus próprios dados

print("🎮 EXERCÍCIO 1: Sistema de Recomendação de Cursos")
print("=" * 55)
print("📝 Sua missão:")
print("1. Criar modelo RecomendacaoCurso com campos relevantes")
print("2. Criar template que recebe interesse, nível e tempo disponível")
print("3. Implementar chain completa")
print("4. Testar com seus dados")
print("\n💡 Dica: Pense nos campos que seriam úteis em uma recomendação real!")
print("\n🚀 Comece aqui embaixo:")

# SEU CÓDIGO AQUI!
# 
# class RecomendacaoCurso(BaseModel):
#     # Defina os campos necessários
#     pass
# 
# # Crie o parser, template e chain
# 
# # Teste o sistema

# EXEMPLO DE SOLUÇÃO (descomente para ver):
"""
class RecomendacaoCurso(BaseModel):
    nome_curso: str = Field(description="Nome do curso recomendado")
    plataforma: str = Field(description="Plataforma onde encontrar")
    duracao_horas: int = Field(description="Duração em horas")
    nivel: str = Field(description="Nível: iniciante, intermediário ou avançado")
    preco: float = Field(description="Preço em reais")
    motivo: str = Field(description="Por que é recomendado")
    habilidades: List[str] = Field(description="Habilidades que vai desenvolver")

parser = PydanticOutputParser(pydantic_object=RecomendacaoCurso)
template = ChatPromptTemplate.from_template(...)
chain = template | llm | parser
"""

print("\n⏰ Tempo sugerido: 15 minutos")
print("🆘 Dúvidas? Chama o Pedro!")

## 🔗 Conectando com o Futuro - Chains

**Preparando o terreno para o próximo módulo! 🚀**

Tudo que vimos hoje são os **blocos fundamentais** para construir **Chains** mais complexas!

**Analogia do LEGO:** 🧱
- **Templates** = Peças básicas coloridas
- **Parsers** = Conectores especiais
- **LCEL** = Instruções de montagem
- **Chains** (próximo módulo) = Construções complexas!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-03_img_06.png)

In [ ]:
# Preview do que vem no próximo módulo: Chains avançadas!
# Vamos criar uma mini-chain que combina múltiplas etapas

print("🔮 PREVIEW: Chains Avançadas (Próximo Módulo)")
print("=" * 50)

# Simulando uma chain com múltiplas etapas
# Etapa 1: Gerar ideia
ideia_template = ChatPromptTemplate.from_template(
    "Sugira uma ideia criativa para um app sobre {tema}"
)

# Etapa 2: Avaliar a ideia (usando nosso parser!)
class AvaliacaoIdeia(BaseModel):
    nome_app: str = Field(description="Nome sugerido para o app")
    viabilidade: int = Field(description="Viabilidade de 1 a 10")
    publico_alvo: str = Field(description="Público-alvo")
    monetizacao: str = Field(description="Como monetizar")

avaliacao_parser = PydanticOutputParser(pydantic_object=AvaliacaoIdeia)
avaliacao_template = ChatPromptTemplate.from_template(
    """
    Avalie esta ideia de app: {ideia}
    
    Seja realista sobre viabilidade técnica e de mercado.
    
    {format_instructions}
    """
)

# Criando as chains
chain_ideia = ideia_template | llm
chain_avaliacao = avaliacao_template | llm | avaliacao_parser

print("🧪 Testando preview de chain complexa...")

try:
    # Etapa 1: Gerar ideia
    ideia = chain_ideia.invoke({"tema": "sustentabilidade"})
    print(f"💡 Ideia gerada: {ideia.content[:100]}...")
    
    # Etapa 2: Avaliar ideia
    avaliacao = chain_avaliacao.invoke({
        "ideia": ideia.content,
        "format_instructions": avaliacao_parser.get_format_instructions()
    })
    
    print(f"\n⭐ Avaliação estruturada:")
    print(f"📱 Nome: {avaliacao.nome_app}")
    print(f"📊 Viabilidade: {avaliacao.viabilidade}/10")
    print(f"🎯 Público: {avaliacao.publico_alvo}")
    print(f"💰 Monetização: {avaliacao.monetizacao}")
    
    print("\n🎉 Isso é só o começo! No próximo módulo vamos aprender a:")
    print("  ✅ Criar chains complexas com múltiplas etapas")
    print("  ✅ Fazer chains condicionais")
    print("  ✅ Chains com loops e decisões")
    print("  ✅ Chains que se corrigem automaticamente")
    
except Exception as e:
    print(f"❌ Erro no preview: {e}")
    print("Não se preocupe, vamos aprender a lidar com isso no próximo módulo!")

## 🎮 Exercício Final - Desafio Completo

**Hora do desafio final! 🏆**

Vamos criar um sistema mais complexo que combina tudo que aprendemos!

In [ ]:
# 🏆 DESAFIO FINAL: Sistema de Análise de Carreira em Tech
# 
# Crie um sistema que:
# 1. Recebe: cargo_atual, anos_experiencia, tecnologias, objetivo
# 2. Retorna análise estruturada com próximos passos
# 3. Inclui roadmap, cursos, salário esperado, etc.

print("🏆 DESAFIO FINAL: Sistema de Análise de Carreira")
print("=" * 50)
print("🎯 Seu objetivo: Criar um consultor de carreira IA!")
print("\n📋 Requisitos:")
print("1. Modelo Pydantic com pelo menos 8 campos relevantes")
print("2. Template que recebe informações da carreira atual")
print("3. Sistema que dá conselhos práticos e realistas")
print("4. Roadmap estruturado com timeline")
print("\n💡 Campos sugeridos:")
print("  - proximo_cargo, salario_esperado, tempo_transicao")
print("  - cursos_recomendados, certificacoes, soft_skills")
print("  - roadmap_6_meses, roadmap_1_ano, dificuldades")

print("\n🚀 Implemente aqui:")

# SEU CÓDIGO AQUI!
# Seja criativo e pense no que seria realmente útil!

# ESTRUTURA SUGERIDA:
# class AnaliseCarreira(BaseModel):
#     # Seus campos aqui
#     pass
# 
# # Parser, template e chain
# 
# # Teste com diferentes perfis

print("\n⏰ Tempo sugerido: 25 minutos")
print("🎁 Bônus: Se conseguir implementar, você já está pronto para o próximo módulo!")

## 📊 Diagrama de Fluxo Completo

Vamos visualizar tudo que aprendemos hoje:

In [ ]:
# Criando um diagrama Mermaid conceitual
print("""
🎯 FLUXO COMPLETO: Templates + Parsers + LCEL

```mermaid
graph TD
    A["🔤 Input Variables"] --> B["📝 Prompt Template"]
    B --> C["✨ Formatted Prompt"]
    C --> D["🤖 LLM (Gemini)"]
    D --> E["📄 Raw Response"]
    E --> F["🔧 Output Parser"]
    F --> G["📊 Structured Data"]
    
    H["💡 Format Instructions"] --> F
    I["🔍 Pydantic Validation"] --> F
    
    G --> J["✅ Ready for Use"]
    
    style A fill:#e1f5fe
    style B fill:#f3e5f5
    style D fill:#fff3e0
    style F fill:#e8f5e8
    style G fill:#fce4ec
```
""")

# Vamos criar uma visualização em Python também
fig, ax = plt.subplots(figsize=(14, 10))

# Definindo posições dos componentes
componentes = {
    'Input\nVariables': (1, 8, 'lightblue'),
    'Prompt\nTemplate': (3, 8, 'lightcyan'),
    'Formatted\nPrompt': (5, 8, 'lightyellow'),
    'LLM\n(Gemini)': (7, 8, 'lightcoral'),
    'Raw\nResponse': (9, 8, 'lightpink'),
    'Output\nParser': (11, 8, 'lightgreen'),
    'Structured\nData': (13, 8, 'lavender'),
    'Format\nInstructions': (11, 6, 'wheat'),
    'Pydantic\nValidation': (11, 4, 'lightsteelblue'),
    'Ready for\nProduction': (13, 6, 'lightseagreen')
}

# Desenhando os componentes
for nome, (x, y, cor) in componentes.items():
    circle = plt.Circle((x, y), 0.8, facecolor=cor, edgecolor='black', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, y, nome, ha='center', va='center', fontsize=9, fontweight='bold')

# Desenhando as conexões
conexoes = [
    ((1, 8), (3, 8)),    # Input -> Template
    ((3, 8), (5, 8)),    # Template -> Formatted
    ((5, 8), (7, 8)),    # Formatted -> LLM
    ((7, 8), (9, 8)),    # LLM -> Raw Response
    ((9, 8), (11, 8)),   # Raw Response -> Parser
    ((11, 8), (13, 8)),  # Parser -> Structured
    ((11, 6), (11, 8)),  # Format Instructions -> Parser
    ((11, 4), (11, 8)),  # Validation -> Parser
    ((13, 8), (13, 6))   # Structured -> Production
]

for (x1, y1), (x2, y2) in conexoes:
    ax.annotate('', xy=(x2-0.3, y2), xytext=(x1+0.3, y1),
                arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))

ax.set_xlim(0, 14)
ax.set_ylim(3, 9)
ax.set_title('🎯 Arquitetura Completa: Templates + Parsers + LCEL\n(O que aprendemos hoje)', 
             fontsize=16, fontweight='bold', pad=20)
ax.axis('off')

plt.tight_layout()
plt.show()

print("\n🎉 Parabéns! Você dominou os fundamentos dos Templates e Parsers!")

## 🎯 Resumo do Módulo 3

**O que você aprendeu hoje? Liiiindo! 🎉**

### 📝 Prompt Templates:
- ✅ **O que são**: Moldes reutilizáveis para prompts
- ✅ **Por que usar**: Consistência, reutilização, manutenção
- ✅ **Tipos**: PromptTemplate, ChatPromptTemplate (recomendado)
- ✅ **Melhores práticas**: Sempre use para projetos sérios

### 🔧 Output Parsers:
- ✅ **O que são**: Organizadores de respostas da IA  
- ✅ **Tipos**: Lista simples, JSON custom, Pydantic (recomendado)
- ✅ **Pydantic**: Validação automática, type hints, robustez
- ✅ **Benefícios**: Dados estruturados, menos bugs, mais produtividade

### 🔗 Integração com LCEL:
- ✅ **Chains simples**: Template | LLM | Parser
- ✅ **Preparação**: Base sólida para chains complexas
- ✅ **Próximo módulo**: Chains avançadas com múltiplas etapas

---

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-03_img_07.png)

## 🚀 Preparando para o Módulo 4: Chains

**O que vem por aí? 🔮**

No próximo módulo vamos aprender:

1. **Sequential Chains**: Chains que executam em sequência
2. **Conditional Chains**: Chains que tomam decisões
3. **Parallel Chains**: Executando múltiplas chains ao mesmo tempo
4. **Error Handling**: Chains que se recuperam de erros
5. **Chain Composition**: Combinando chains complexas

### 💡 Dica do Pedro Final:

**Para se preparar para o próximo módulo:**
- ✅ Pratique criando templates diversos
- ✅ Experimente diferentes parsers
- ✅ Combine templates + parsers + LCEL
- ✅ Pense em casos de uso do seu trabalho

**Lição de casa (opcional):** 📚
Tente criar um sistema completo usando tudo que aprendemos hoje. Pode ser:
- Sistema de análise de currículos
- Gerador de relatórios estruturados  
- Assistente de planejamento de viagens
- Analisador de sentimentos com dados estruturados

---

## 🎊 Parabéns!

Você concluiu o **Módulo 3** do curso de LangChain!

Agora você tem as ferramentas fundamentais para criar sistemas de IA profissionais e estruturados. No próximo módulo, vamos usar tudo isso para construir **Chains** ainda mais poderosas!

**Nos vemos no Módulo 4! Bora para as Chains! 🚀**

---

*"O sucesso é a soma de pequenos esforços repetidos dia após dia."* 💪

**#LangChain #IA #Python #PedroGuth**